# 1. Weight and Bias Login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_04"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

# 2. Login Huggingface

In [2]:
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 3. Dataset Load

In [3]:
from datasets import load_dataset

In [4]:
ko_instruction_01 = load_dataset("nlpai-lab/kullm-v2", cache_dir="/mnt/t7/.cache/huggingface/datasets", split="train")

In [5]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=210)

In [6]:
ko_instruction_01

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})

In [7]:
ko_instruction_01 = ko_instruction_01.train_test_split(test_size=0.97)

In [8]:
ko_instruction_01["train"][100]

{'id': 'vicuna_{idx}',
 'instruction': '통합 빌드 단계를 기술적으로 세분화해 보세요. 모든 프로그램이 열려 있고 빌드할 준비가 되었다고 가정해 보겠습니다.',
 'input': '',
 'output': 'Greenhouse ATS와 BambooHR 간의 통합을 구축하려면 다음 단계를 따라야 합니다:1. Boomi 플랫폼에서 "생성" 버튼을 클릭하고 드롭다운 메뉴에서 "프로세스"를 선택합니다. 그러면 통합을 생성할 수 있는 프로세스 빌더가 열립니다.2. 프로세스 빌더에서 "커넥터 추가" 버튼을 클릭하고 목록에서 Greenhouse ATS 및 BambooHR 커넥터를 선택합니다. 그러면 캔버스에 커넥터가 추가되어 이러한 시스템의 데이터에 액세스할 수 있습니다.3. 다음으로 두 시스템 간에 동기화할 데이터를 지정해야 합니다. 이렇게 하려면 Greenhouse ATS 커넥터에서 적절한 필드를 캔버스로 끌어다 놓은 다음 BambooHR 커넥터의 해당 필드에 매핑하면 됩니다. 이렇게 하면 두 시스템 간에 데이터가 올바르게 동기화됩니다.4. 필드를 매핑한 후에는 프로세스 빌더를 사용하여 통합을 위한 워크플로를 만들 수 있습니다. 여기에는 데이터가 올바르게 동기화되도록 적절한 트리거, 작업 및 조건을 설정하는 것이 포함됩니다. 예를 들어, Greenhouse ATS에서 오퍼 세부 정보가 변경될 때 실행되는 트리거를 설정하고, BambooHR에서 해당 정보를 업데이트하는 작업을 설정할 수 있습니다.5. 통합 구축을 완료한 후에는 통합이 올바르게 작동하는지 테스트할 수 있습니다. 프로세스 빌더의 테스트 기능을 사용하면 통합을 시뮬레이션하고 실제로 어떻게 작동하는지 보여줄 수 있습니다. 모든 것이 정상적으로 보이면 통합을 저장하고 Boomi 환경에 배포할 수 있습니다.6. 통합을 배포한 후에는 자동으로 실행이 시작되고 설정한 워크플로우에 따라 Greenhouse ATS와 BambooHR 간에 데이터가 동기화됩니다. 통합을 모니터링하여 올바르게 작

# 4. Formatting Dataset

In [9]:
def format_instruction(sample):
    system_prompt = f"### instruction: {sample['instruction']}"
    input = f"### input: {sample['input']}" if len(sample["input"]) > 0 else None
    output = f"### output: {sample['output']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}"
    return sample

In [10]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/4578 [00:00<?, ? examples/s]

In [11]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 4578
})

In [12]:
train_dataset["text"][100]

'### instruction: 통합 빌드 단계를 기술적으로 세분화해 보세요. 모든 프로그램이 열려 있고 빌드할 준비가 되었다고 가정해 보겠습니다.\n\n### output: Greenhouse ATS와 BambooHR 간의 통합을 구축하려면 다음 단계를 따라야 합니다:1. Boomi 플랫폼에서 "생성" 버튼을 클릭하고 드롭다운 메뉴에서 "프로세스"를 선택합니다. 그러면 통합을 생성할 수 있는 프로세스 빌더가 열립니다.2. 프로세스 빌더에서 "커넥터 추가" 버튼을 클릭하고 목록에서 Greenhouse ATS 및 BambooHR 커넥터를 선택합니다. 그러면 캔버스에 커넥터가 추가되어 이러한 시스템의 데이터에 액세스할 수 있습니다.3. 다음으로 두 시스템 간에 동기화할 데이터를 지정해야 합니다. 이렇게 하려면 Greenhouse ATS 커넥터에서 적절한 필드를 캔버스로 끌어다 놓은 다음 BambooHR 커넥터의 해당 필드에 매핑하면 됩니다. 이렇게 하면 두 시스템 간에 데이터가 올바르게 동기화됩니다.4. 필드를 매핑한 후에는 프로세스 빌더를 사용하여 통합을 위한 워크플로를 만들 수 있습니다. 여기에는 데이터가 올바르게 동기화되도록 적절한 트리거, 작업 및 조건을 설정하는 것이 포함됩니다. 예를 들어, Greenhouse ATS에서 오퍼 세부 정보가 변경될 때 실행되는 트리거를 설정하고, BambooHR에서 해당 정보를 업데이트하는 작업을 설정할 수 있습니다.5. 통합 구축을 완료한 후에는 통합이 올바르게 작동하는지 테스트할 수 있습니다. 프로세스 빌더의 테스트 기능을 사용하면 통합을 시뮬레이션하고 실제로 어떻게 작동하는지 보여줄 수 있습니다. 모든 것이 정상적으로 보이면 통합을 저장하고 Boomi 환경에 배포할 수 있습니다.6. 통합을 배포한 후에는 자동으로 실행이 시작되고 설정한 워크플로우에 따라 Greenhouse ATS와 BambooHR 간에 데이터가 동기화됩니다. 통합을 모니터링하여 올바르게 작동하는지 확인하고 필요한 경우 필요한 조정을 수행할 수 있습니다

# 5. Loading the Model

In [13]:
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [15]:
# 4bit QLoRA 학습을 위한 설정
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

In [16]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map=device_map, cache_dir=cache_model_dir, trust_remote_code=True)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, cache_dir=cache_model_dir)
tokenizer.pad_token = tokenizer.eos_token

# 6. LoRA Setup

In [18]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

In [19]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

# 7. Training Argument Setup

In [20]:
from transformers import TrainingArguments

In [21]:
output_dir = "./checkpoint/experi_05"
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
report_to="wandb"
save_steps = 20
save_total_limit=5
num_train_epochs = 3
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [22]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    logging_steps=logging_steps,
    report_to = report_to,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [23]:
from trl import SFTTrainer

max_seq_length = 1024

In [24]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/4578 [00:00<?, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [25]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# 8. Training

In [26]:
trainer.train()

Step,Training Loss
10,2.306900
20,2.374600
30,2.915500
40,3.037900
50,4.086000
60,2.004700
70,2.296300
80,2.120000
90,2.257700
100,2.141400


TrainOutput(global_step=6867, training_loss=1.6604569136393337, metrics={'train_runtime': 7186.9162, 'train_samples_per_second': 1.911, 'train_steps_per_second': 0.955, 'total_flos': 1.988202042566738e+17, 'train_loss': 1.6604569136393337, 'epoch': 3.0})

In [27]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [28]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [30]:
text = "### instruction: AI의 정의에 대해 설명해줘.\n\n### output:"

In [31]:
device = "cuda:0"

In [32]:
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=500)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [33]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### instruction: AI의 정의에 대해 설명해줘.

### output: AI는 인공지능을 말합니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​


In [34]:
print(len(outputs[0]))

520


In [36]:
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s> ### instruction: AI의 정의에 대해 설명해줘.

### output: AI는 인공지능을 말합니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​


In [39]:
outputs[0]

tensor([    1,   835, 15278, 29901,   319, 29902, 30708, 32984, 31054, 32550,
        32750, 31435, 45189, 29889,    13,    13,  2277, 29937,  1962, 29901,
          319, 29902, 31081, 37651, 31286, 32047, 32111, 29889, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 

In [37]:
tokenizer.eos_token

'</s>'

In [38]:
tokenizer.eos_token_id

2

In [40]:
pad_tok = [30166]

In [41]:
tokenizer.decode(pad_tok, skip_special_tokens=False)

'\u200b'

In [46]:
text_01 = "### instruction: 1+2의 값이 뭐야?.\n\n### output:"

In [47]:
inputs = tokenizer(text_01, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=500)

In [48]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(len(outputs[0]))

### instruction: 1+2의 값이 뭐야?.

### output: 3+4의 값이 뭐야?.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​
519


In [54]:
pad_tok = [38535]

In [55]:
tokenizer.decode(pad_tok, skip_special_tokens=False)

'뭐야'

In [49]:
outputs[0]

tensor([    1,   835, 15278, 29901, 29871, 29896, 29974, 29906, 30708, 36663,
        30393, 38535,  9808,    13,    13,  2277, 29937,  1962, 29901, 29871,
        29941, 29974, 29946, 30708, 36663, 30393, 38535,  9808, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166,
        30166, 30166, 30166, 30166, 30166, 30166, 30166, 30166, 

In [57]:
text_02 = "### instruction: 에디오피아에 대해 알려줘.\n\n### output:"

In [58]:
inputs = tokenizer(text_02, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=500)

In [59]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(len(outputs[0]))

### instruction: 에디오피아에 대해 알려줘.

### output: 에디오피아는 아프리카에 있는 나라야. 아프리카에는 54개의 나라가 있어. 에디오피아는 그 중 하나야. 에디오피아는 아프리카에서 가장 큰 나라야. 에디오피아는 세계에서 10번째로 큰 나라야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 가장 큰 커피 수출국이야. 에디오피아는 세계에서 가장 큰 커피 소비국이야. 에디오피아는 세계에서 가장 큰 커피 생산국이야. 에디오피아는 세계에서 

In [60]:
text_03 = "### instruction: 작업치료사가 어떤 치료를 하는지 알려줘.\n\n### output:"

In [61]:
inputs = tokenizer(text_03, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=500)

In [62]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(len(outputs[0]))

### instruction: 작업치료사가 어떤 치료를 하는지 알려줘.

### output: 작업치료사는 환자의 일상생활을 돕고, 환자의 신체적, 정신적, 사회적, 직업적, 심리적, 영적 기능을 향상시키기 위해 치료적 개입을 한다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​
519


In [63]:
model_to_save.save_pretrained("./results/experi_05")

In [64]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(46336, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_feature

In [65]:
model = model.merge_and_unload()

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:272: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [66]:
output_merged_dir = "./lora_merged_model/sft_05"
model.save_pretrained(output_merged_dir)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 4096}


In [67]:
model.push_to_hub('aeolian83/llama_ko_sft_kullm_experi_02')
tokenizer.push_to_hub('aeolian83/llama_ko_sft_kullm_experi_02')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 4096}


model-00002-of-00002.safetensors:   0%|          | 0.00/759M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aeolian83/llama_ko_sft_kullm_experi_02/commit/d4fd15c40eedac94b5e246d6f2a04ad0daeb8f80', commit_message='Upload tokenizer', commit_description='', oid='d4fd15c40eedac94b5e246d6f2a04ad0daeb8f80', pr_url=None, pr_revision=None, pr_num=None)